In [1]:
import omicverse as ov
import scanpy as sc
import matplotlib.pyplot as plt
ov.ov_plot_set()
import pandas as pd
import anndata as ad
import numpy as np
from matplotlib import patheffects

/home/zoo/ball6395/software/miniconda3/envs/omicverse/lib/python3.10/torch_geometric/typing.py:54: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.27' not found (required by /home/zoo/ball6395/software/miniconda3/envs/omicverse/lib/python3.10/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/home/zoo/ball6395/software/miniconda3/envs/omicverse/lib/python3.10/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.27' not found (required by /home/zoo/ball6395/software/miniconda3/envs/omicverse/lib/python3.10/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "



   ____            _     _    __                  
  / __ \____ ___  (_)___| |  / /__  _____________ 
 / / / / __ `__ \/ / ___/ | / / _ \/ ___/ ___/ _ \ 
/ /_/ / / / / / / / /__ | |/ /  __/ /  (__  )  __/ 
\____/_/ /_/ /_/_/\___/ |___/\___/_/  /____/\___/                                              

Version: 1.6.3, Tutorials: https://omicverse.readthedocs.io/
All dependencies are satisfied.


In [2]:
np.random.seed(42)

In [6]:
path = "../2.SAM_reclustering/SAM/"

In [7]:
Hsap = sc.read_h5ad(f'{path}/Hsap_Oligo_OPC.15.30.2500.DonorID.h5ad')
Mmus = sc.read_h5ad(f'{path}/Mmus_Oligo_OPC.15.30.2500.DonorID.h5ad')
Pvit = sc.read_h5ad(f'{path}/Pvit_Oligo_OPC.15.30.2500.DonorID.h5ad')

In [8]:
# modify for plotting
# Hsap
Hsap.obs['location'] = [i.split(' - ')[0] for i in Hsap.obs['Tissue']]

Hsap.obs['location'][Hsap.obs['location'].isin(['Head of hippocampus (HiH)',  'Amygdaloid complex (AMY)', 'Extended amygdala (EXA)',
                                    'Body of hippocampus (HiB)', 'Basal nuclei (BN)', 'Tail of Hippocampus (HiT)','Claustrum', 'Paleocortex (PalCx)',
                                    'Basal forebrain (BF)','Cerebral cortex (Cx)', 'Perirhinal cortex (area 35) (A35)'])] = 'Telencephalon'
Hsap.obs['location'][Hsap.obs['location'].isin(['Thalamus (THM)', 'Hypothalamus (HTH)', 'Epithalamus'])] = 'Diencephalon'
Hsap.obs['location'][Hsap.obs['location'].isin(['Midbrain (M)','Midbrain (RN)'])] = 'Mesencephalon'
Hsap.obs['location'][Hsap.obs['location'].isin(['Myelencephalon (medulla oblongata) (Mo)','Pons (Pn)', 'Cerebellum (CB)'])] = 'Rhombencephalon'
# Mmus
Mmus.obs['location'] = Mmus.obs['Tissue'].astype('string')
Mmus.obs['location'][Mmus.obs['location'].isin(['StriatDor', 'StriatVent', 'Amygd', 'HC', 'OB', 'CA1',
                                                'Ctx1','Ctx3','Ctx2','Ctx1.5','DentGyr','SScortex'])] = 'Telencephalon'
Mmus.obs['location'][Mmus.obs['location'].isin(['Thal', 'Hypoth'])] = 'Diencephalon'
Mmus.obs['location'][Mmus.obs['location'].isin(['MBd', 'MBv', ])] = 'Mesencephalon'
Mmus.obs['location'][Mmus.obs['location'].isin(['Medulla','CB'])] = 'Rhombencephalon'
# Pvit
Pvit.obs['location'] = Pvit.obs['Region'].astype('string')
Pvit.obs['location'][Pvit.obs['location'].isin(['Telencephalon', 'amDVR'])] = 'Telencephalon'
Pvit.obs['location'][Pvit.obs['location'].isin(['Thalamus+Hypothalamus'])] = 'Diencephalon'
Pvit.obs['location'][Pvit.obs['location'].isin(['Thalamus+Hypothalamus+Midbrain',
                                               'Optic Tectum + Midbrain', 'Optic Tectum'])] = 'Di-/Mesencephalon'
Pvit.obs['location'][Pvit.obs['location'].isin(['Cerebellum', 'posterior brain', 'Brainstem'])] = 'Rhombencephalon'

In [9]:
color_dict = {'Telencephalon':'#DE582B', 'Diencephalon':'#A4E048', 'Di-/Mesencephalon':'#738061',
              'Mesencephalon':'#808080','Rhombencephalon':'#293890', 'Others': '#000000'}
ov.pl.embedding(
    Mmus, basis = 'X_umap', color = ['location'], show = False, palette=color_dict, frameon='small', save= ".Mmus_Oligo_OPC.pdf", size=2)
ov.pl.embedding(
    Hsap, basis = 'X_umap', color = ['location'], show = False, palette=color_dict, frameon='small', save= ".Hsap_Oligo_OPC.pdf", size=2)
ov.pl.embedding(
    Pvit, basis = 'X_umap', color = ['location'], show = False, palette=color_dict, frameon='small', save= ".Pvit_Oligo_OPC.pdf", size=2)

<AxesSubplot: title={'center': 'location'}, xlabel='X_umap1', ylabel='X_umap2'>

In [12]:
meta_path = "."
umap = pd.read_csv(f'{meta_path}/Amniote_Oligo_OPC.UMAP.csv', index_col=0)
meta = pd.read_csv(f'{meta_path}/Amniote_Oligo_OPC.meta.csv', index_col=0)

Hsap.obsm['X_umap'] = umap.loc[Hsap.obs.index].values
Mmus.obsm['X_umap'] = umap.loc[Mmus.obs.index].values
Pvit.obsm['X_umap'] = umap.loc[Pvit.obs.index].values

Hsap.obs['Oligo_OPC.integrated_leiden.1'] = meta.loc[Hsap.obs.index]['Oligo_OPC.integrated_leiden.1'].astype('string')
Mmus.obs['Oligo_OPC.integrated_leiden.1'] = meta.loc[Mmus.obs.index]['Oligo_OPC.integrated_leiden.1'].astype('string')
Pvit.obs['Oligo_OPC.integrated_leiden.1'] = meta.loc[Pvit.obs.index]['Oligo_OPC.integrated_leiden.1'].astype('string')

In [13]:
vertebrate = ad.concat([Hsap, Mmus, Pvit], join='outer', axis=0)

In [14]:
vertebrate.obs['Species'] =  pd.Categorical(vertebrate.obs['Species'], categories=['Hsap','Mmus','Pvit'], ordered=True)

In [15]:
vertebrate.obs['Species'].value_counts()

Hsap    67691
Pvit    42881
Mmus    22660
Name: Species, dtype: int64

In [16]:
# subset some cells for balancing number of cells in different species, just for better visualisation
ID1 = vertebrate.obs[~vertebrate.obs['Species'].isin(['Hsap'])].sample(20000, random_state=42).index.tolist()
ID2 = vertebrate.obs[~vertebrate.obs['Species'].isin(['Mmus'])].sample(20000, random_state=42).index.tolist()
ID3 = vertebrate.obs[~vertebrate.obs['Species'].isin(['Pvit'])].sample(20000, random_state=42).index.tolist()
vertebrate = vertebrate[vertebrate.obs.index.isin(list(ID1+ID2+ID3))].copy()

In [17]:
color_dict = {'Hsap':'#989A9C', 'Mmus':'#F7D08D','Pvit':'#BF83A5','Pmar':'#8684B0'}
# plot species factors in neuorns and non-neurons atlas
fig, ax = plt.subplots(figsize=(4,4))
random_indices = np.random.permutation(list(range(vertebrate.shape[0])))
ov.pl.embedding(
    vertebrate[random_indices, :],
    basis="X_umap",
    color=['Species'],
    title='',
    show=False,  
    size=2,
    palette=color_dict,
    frameon='small',
    ax=ax, 
    save= ".vertebrate_Oligo_OPC.species.pdf"
)

<AxesSubplot: xlabel='X_umap1', ylabel='X_umap2'>

In [18]:
color_dict = {'Telencephalon':'#DE582B', 'Diencephalon':'#A4E048', 'Di-/Mesencephalon':'#738061',
              'Mesencephalon':'#808080','Rhombencephalon':'#293890', 'Others': '#000000'}# plot species factors in neuorns and non-neurons atlas
fig, ax = plt.subplots(figsize=(4,4))
random_indices = np.random.permutation(list(range(vertebrate.shape[0])))
ov.pl.embedding(
    vertebrate[random_indices, :],
    basis="X_umap",
    color=['location'],
    title='',
    show=False,  
    size=2,
    palette=color_dict,
    frameon='small',
    ax=ax, 
    save= ".vertebrate_Oligo_OPC.region.pdf"
)

<AxesSubplot: xlabel='X_umap1', ylabel='X_umap2'>

In [20]:
fig, ax = plt.subplots(figsize=(4,4))
random_indices = np.random.permutation(list(range(vertebrate.shape[0])))
ov.pl.embedding(
    vertebrate[random_indices, :],
    basis="X_umap",
    color=['Oligo_OPC.integrated_leiden.1'],
    title='',
    show=False,  
    size=2,
    frameon='small',
    ax=ax, 
    save= ".vertebrate_Oligo_OPC.subtype.pdf"
)

<AxesSubplot: xlabel='X_umap1', ylabel='X_umap2'>